In [18]:
import os
import xml

In [19]:
pos_dimers = !ls positive_dimers/predictions/
neg_dimers = !ls negative_dimers/predictions/

pos_dimers[:10]

['O00189+Q9UPM8',
 'O00212+Q9HAD4',
 'O00541+Q14684',
 'O14544+Q86XK2',
 'O14641+P10515',
 'O14641+Q13501',
 'O14976+Q13470',
 'O15294+P60228',
 'O43264+Q9UFG5',
 'O43402+P36776']

In [4]:
if not os.path.isdir("interfaces/"):
    os.makedirs("interfaces/positive")
    os.makedirs("interfaces/negative")
for dimer in pos_dimers:
    pos_dimers_path = f"positive_dimers/predictions/{dimer}/ranked_0.pdb"
    cmd1 = f"pisa {dimer} -analyse {pos_dimers_path}"
    cmd2 = f"pisa {dimer} -xml interfaces > interfaces/positive/{dimer}.xml"
    cmd3 = f"pisa {dimer} -erase"
    os.system(cmd1)
    os.system(cmd2)
    os.system(cmd3)


 ... session 'O00189+Q9UPM8' Created
 ... structure read and understood
 ... structural equivalence analysed
 ... molecular surfaces analysed
 ... 1 potential contacts identified, now processing:
     |
 ... total 1 interfaces calculated
 ... assembly analysis: not performed as no crystal data were found.
 ... results written

 +++ session 'O00189+Q9UPM8' erased


 ... session 'O00212+Q9HAD4' Created
 ... structure read and understood
 ... structural equivalence analysed
 ... molecular surfaces analysed
 ... 1 potential contacts identified, now processing:
     |
 ... total 1 interfaces calculated
 ... assembly analysis: not performed as no crystal data were found.
 ... results written

 +++ session 'O00212+Q9HAD4' erased


 ... session 'O00541+Q14684' Created
 ... structure read and understood
 ... structural equivalence analysed
 ... molecular surfaces analysed
 ... 1 potential contacts identified, now processing:
     |
 ... total 1 interfaces calculated
 ... assembly analysis: not

In [6]:
if not os.path.isdir("interfaces/negative"):
    os.makedirs("interfaces/negative")
for dimer in neg_dimers:
    neg_dimers_path = f"negative_dimers/predictions/{dimer}/ranked_0.pdb"
    cmd1 = f"pisa {dimer} -analyse {neg_dimers_path}"
    cmd2 = f"pisa {dimer} -xml interfaces > interfaces/negative/{dimer}.xml"
    cmd3 = f"pisa {dimer} -erase"
    os.system(cmd1)
    os.system(cmd2)
    os.system(cmd3)


 ... session 'A1X283+Q13618' Created
 ... structure read and understood
 ... structural equivalence analysed
 ... molecular surfaces analysed
 ... 1 potential contacts identified, now processing:
     |
 ... total 1 interfaces calculated
 ... assembly analysis: not performed as no crystal data were found.
 ... results written

 +++ session 'A1X283+Q13618' erased


 ... session 'O00212+O43663' Created
 ... structure read and understood
 ... structural equivalence analysed
 ... molecular surfaces analysed
 ... 1 potential contacts identified, now processing:
     |
 ... total 1 interfaces calculated
 ... assembly analysis: not performed as no crystal data were found.
 ... results written

 +++ session 'O00212+O43663' erased


 ... session 'O14733+Q6PI77' Created
 ... structure read and understood
 ... structural equivalence analysed
 ... molecular surfaces analysed
 ... 1 potential contacts identified, now processing:
     |
 ... total 1 interfaces calculated
 ... assembly analysis: not

In [20]:
def parse_pisa_xml_outfle(xml_fle,
                         outfle_name=None):
    import xml.etree.ElementTree as ET
    intf_area = ''
    pvalue = ''
    energy = ''
    hb = ''
    sb = ''
    dict_pisa = {}
    if outfle_name == None:
        outfle_name = xml_fle.split('_pisa')[0] + '_dict_pisa.json'
    try:
        root = ET.parse(xml_fle).getroot()
        for interface in root.findall('interface'):
            intf_ch = []
            intf_area = "%.2f" % float(interface.find('int_area').text)
            pvalue = "%.2f" % float(interface.find('pvalue').text)
            energy = "%.2f" % float(interface.find('int_solv_en').text)
            for mol in interface.findall('molecule'):
                mol1 = mol.find('chain_id').text
                intf_ch.append(mol1)
            intf_chains = '_'.join(intf_ch)
            dict_pisa[intf_chains] = {}
            for hbonds in interface.findall('h-bonds'):
                hb = int(hbonds.find('n_bonds').text)
            for sb in interface.findall('salt-bridges'):
                sb = int(sb.find('n_bonds').text)
            dict_pisa[intf_chains]['int_area'] = intf_area
            dict_pisa[intf_chains]['pvalue'] = pvalue
            dict_pisa[intf_chains]['int_solv_en'] = energy
            dict_pisa[intf_chains]['hb'] = hb
            dict_pisa[intf_chains]['sb'] = sb
    except:
        root = []
        pass
    if dict_pisa:
        with open(outfle_name,'w') as outfle:
            json.dump(dict_pisa,outfle)
    return dict_pisa


In [27]:
import json
import pandas as pd

rows = []
for dimer in pos_dimers:
    pos_xml_path = f"interfaces/positive/{dimer}.xml"
    dict_pisa = parse_pisa_xml_outfle(pos_xml_path)
    for interface, metrics in dict_pisa.items():
        row = {"complex": dimer, "interface_name": interface, **metrics}
        rows.append(row)
df = pd.DataFrame(rows)
# make numeric columns numeric (keeps strings if they aren’t)
for col in df.columns:
    if col != "interface_name" and col != "complex":
        df[col] = pd.to_numeric(df[col], errors="ignore")

df.to_csv("positive_interfaces.csv", index=False)

/tmp/ipykernel_788663/74923332.py:15: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors="ignore")


In [29]:
import json
import pandas as pd

rows = []
for dimer in neg_dimers:
    neg_xml_path = f"interfaces/negative/{dimer}.xml"
    dict_pisa = parse_pisa_xml_outfle(neg_xml_path)
    for interface, metrics in dict_pisa.items():
        row = {"complex": dimer, "interface_name": interface, **metrics}
        rows.append(row)
df = pd.DataFrame(rows)
# make numeric columns numeric (keeps strings if they aren’t)
for col in df.columns:
    if col != "interface_name" and col != "complex":
        df[col] = pd.to_numeric(df[col], errors="ignore")

df.to_csv("negative_interfaces.csv", index=False)

/tmp/ipykernel_788663/909894959.py:15: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors="ignore")


In [32]:
import os, subprocess
import pandas as pd

# 1) resolve sc command (ccp4 env should be loaded)
try:
    import setup_names
    SC_CMD = setup_names.sc_cmd
except Exception:
    SC_CMD = "sc"  # assumes sc is on PATH

def run_sc_once(pdbfile: str, chain_lst=("A","B")) -> dict:
    """Run CCP4 sc for two chains and parse Sc value from output."""
    script = f"{os.path.basename(pdbfile)}_{chain_lst[0]}_{chain_lst[1]}_sc.sh"
    with open(script, "w") as out:
        out.write(f"{SC_CMD} XYZIN {pdbfile} <<eof\n")
        out.write(f"MOLECULE 1\nCHAIN {chain_lst[0]}\nMOLECULE 2\nCHAIN {chain_lst[1]}\nEND\n")
        out.write("eof\n")
    os.chmod(script, 0o755)

    with open("tmp_sc.out", "w") as log:
        subprocess.run(["bash", "-lc", f"./{script}"], check=True,
                       stdout=log, stderr=subprocess.STDOUT)

    pdb_seen, sc_val = None, None
    with open("tmp_sc.out") as fh:
        for line in fh:
            if "Logical name: XYZIN  File name:" in line:
                pdb_seen = line.split()[-1].split("_clean")[0]
            if "Shape complementarity statistic Sc" in line:
                sc_val = line.split("=")[-1].strip()

    return {
        "pdb": pdb_seen or os.path.splitext(os.path.basename(pdbfile))[0],
        "interface_name": f"{chain_lst[0]}_{chain_lst[1]}",
        "sc": sc_val
    }

def collect_sc(dimers, root_prefix: str, label: str) -> pd.DataFrame:
    """Build paths on the fly and run sc where ranked_0.pdb exists."""
    rows = []
    for d in dimers:
        pdb_path = f"{root_prefix}/predictions/{d}/ranked_0.pdb"
        if not os.path.exists(pdb_path):
            # quietly skip missing predictions
            continue
        try:
            rec = run_sc_once(pdb_path, ("A", "B"))
            rec["dimer"] = d
            rec["set"] = label
            rows.append(rec)
        except subprocess.CalledProcessError:
            # sc failed; record a row with NaN sc so you can inspect later
            rows.append({"pdb": os.path.basename(pdb_path),
                         "interface_name": "A_B", "sc": None,
                         "dimer": d, "set": label})
    df = pd.DataFrame(rows)
    if not df.empty:
        df["sc"] = pd.to_numeric(df["sc"], errors="coerce")
        df = df[["interface_name", "pdb", "sc", "dimer", "set"]]
    return df

# ---- run for positives and negatives ----
pos_df = collect_sc(pos_dimers, "positive_dimers", "positive")
neg_df = collect_sc(neg_dimers, "negative_dimers", "negative")

out = pd.concat([pos_df, neg_df], ignore_index=True)
out.to_csv("sc_interfaces.csv", index=False)
print(f"wrote sc_interfaces.csv with {len(out)} rows")


wrote sc_interfaces.csv with 452 rows
